<a href="https://colab.research.google.com/github/karndeepsingh/spacy_binary_converter/blob/main/spacy_formate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U spacy

     |████████████████████████████████| 5.8 MB 34.1 MB/s 
     |████████████████████████████████| 10.1 MB 36.8 MB/s 
     |████████████████████████████████| 623 kB 55.1 MB/s 
     |████████████████████████████████| 456 kB 57.5 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [ ]:
!pip install -U pip setuptools wheel

     |████████████████████████████████| 1.6 MB 24.4 MB/s 
     |████████████████████████████████| 816 kB 54.5 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.1.2                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.11                        
Pipelines        en_core_web_sm (3.1.0)        



In [ ]:
!git clone https://github.com/walidamamou/relation_extraction_transformer.git

Cloning into 'relation_extraction_transformer'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 34 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [ ]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.6 MB 74 kB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
!pip install typer

In [ ]:
import json

import typer
from pathlib import Path

from spacy.tokens import Span, DocBin, Doc
from spacy.vocab import Vocab
from wasabi import Printer
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.util import compile_infix_regex
import re
import spacy

nlp = spacy.blank("en")
# Create a blank Tokenizer with just the English vocab

msg = Printer()

SYMM_LABELS = ["Binds"]
MAP_LABELS = {
    "DEGREE_IN": "DEGREE_IN",
    "EXPERIENCE_IN": "EXPERIENCE_IN"
}

ann = "/content/relation_extraction_transformer/relations_training.txt"
train_file='/content/sample_data/relations_training.spacy'
dev_file='/content/sample_data/relations_dev.spacy'
test_file='/content/sample_data/relations_test.spacy'

In [ ]:
Doc.set_extension("rel", default={},force=True)
vocab = Vocab()

docs = {"train": [], "dev": [], "test": [], "total": []}
ids = {"train": set(), "dev": set(), "test": set(), "total":set()}
count_all = {"train": 0, "dev": 0, "test": 0,"total": 0}
count_pos = {"train": 0, "dev": 0, "test": 0,"total": 0}
with open("/content/relation_extraction_transformer/relations_training.txt", encoding="utf8") as jsonfile:
    file = json.load(jsonfile)
# for example in file:
    example = file[0]
    span_starts = set()
    neg = 0
    pos = 0
            # Parse the tokens
    tokens=nlp(example["document"])    

    spaces=[]
    spaces = [True if tok.whitespace_ else False for tok in tokens]
    words = [t.text for t in tokens]
    doc = Doc(nlp.vocab, words=words, spaces=spaces)


    # # Parse the GGP entities
    spans = example["tokens"]

    entities = []
    span_end_to_start = {}
    for span in spans:
        # print(span)
        entity = doc.char_span(
              span["start"], span["end"], label=span["entityLabel"]
          )
        # print(entity)


        span_end_to_start[span["token_start"]] = span["token_start"]
        # print(span_end_to_start)
        entities.append(entity)
        span_starts.add(span["token_start"])

    doc.ents = entities
    print(doc.ents)
                # Parse the relations
    rels = {}
    for x1 in span_starts:
        for x2 in span_starts:
            rels[(x1, x2)] = {}
    relations = example["relations"]
    print(rels)
    for relation in relations:
            # the 'head' and 'child' annotations refer to the end token in the span
            # but we want the first token
            start = span_end_to_start[relation["head"]]
            end = span_end_to_start[relation["child"]]
            label = relation["relationLabel"]
            # print(rels[(start, end)])
            # print(label)
            #label = MAP_LABELS[label]
            if label not in rels[(start, end)]:
                rels[(start, end)][label] = 1.0
                pos += 1
                # print(pos)
                # print(rels[(start, end)])

                # The annotation is complete, so fill in zero's where the data is missing
    for x1 in span_starts:
        for x2 in span_starts:
            for label in MAP_LABELS.values():
                if label not in rels[(x1, x2)]:
                    neg += 1
                    rels[(x1, x2)][label] = 0.0

                    # print(rels[(x1, x2)])
    doc._.rel = rels
    print(rels)
                # only keeping documents with at least 1 positive case
    if pos > 0:
            docs["total"].append(doc)
            count_pos["total"] += pos
            count_all["total"] += pos + neg
print(docs["total"])            
docbin = DocBin(docs = docs["total"], store_user_data=True)
docbin.to_disk(train_file)
msg.info(
    f"{len(docs['total'])} training sentences"
)


# main(ann, train_file, dev_file, test_file)


(MS/Ph.D., CS/EE)
{(0, 0): {}, (0, 6): {}, (6, 0): {}, (6, 6): {}}
{(0, 0): {'DEGREE_IN': 0.0, 'EXPERIENCE_IN': 0.0}, (0, 6): {'DEGREE_IN': 1.0, 'EXPERIENCE_IN': 0.0}, (6, 0): {'DEGREE_IN': 0.0, 'EXPERIENCE_IN': 0.0}, (6, 6): {'DEGREE_IN': 0.0, 'EXPERIENCE_IN': 0.0}}
[MS/Ph.D. in CS/EE or related areas is required. Strong ability and effectiveness working in a significant technical problem domain, in the term of plan, design, execution, continuous release and service operation. Strong software engineering fundamentals, including coding, problem solving and data analysis skills. Background in machine learning/deep learning (strongly preferred). Passionate and self-motivated. Ability to effectively work in collaborative multiple project team environment and ship production features in a fast-paced environment. Good communication skills, both verbal and written. Customer/end result/Metrics driven in design and development. Strong ability in self-learning, entering new domain, managing thr

In [ ]:
import json

import typer
from pathlib import Path

from spacy.tokens import Span, DocBin, Doc
from spacy.vocab import Vocab
from wasabi import Printer
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.util import compile_infix_regex
import re
import spacy

nlp = spacy.blank("en")
# Create a blank Tokenizer with just the English vocab
msg = Printer()

SYMM_LABELS = ["Binds"]
MAP_LABELS = {
    "ROLE": "ROLE",
    "ADDRESS": "ADDRESS"
}

ann = "/content/project-3-at-2021-09-06-15-50-79eb1b47.json"
train_file='./relations_training.spacy'
dev_file='./relations_dev.spacy'
test_file='./relations_test.spacy'
# def main(json_loc: Path, train_file: Path, dev_file: Path, test_file: Path):
Doc.set_extension("rel", default={},force=True)
vocab = Vocab()

docs = {"train": [], "dev": [], "test": [], "total": []}
ids = {"train": set(), "dev": set(), "test": set(), "total":set()}
count_all = {"train": 0, "dev": 0, "test": 0,"total": 0}
count_pos = {"train": 0, "dev": 0, "test": 0,"total": 0}

with open("/content/project-3-at-2021-09-06-15-50-79eb1b47.json") as jsonfile:
    file = json.load(jsonfile)
    for example in file:
        span_starts = set()
        neg = 0
        pos = 0
        # Parse the tokens
        
        data = example["data"]
        print(main_doc)
        tokens=nlp(data["text"])
        main_doc_index= {token.text : token.i for token in tokens}
        
        # token_start = main_doc_index[tokens[0].text]
        # token_end = main_doc_index[tokens[0].text] + len_tokens
        # print(main_doc_index)
        # print(token_start, token_end)
        annotation = example["annotations"]
        
        
        
          

        spaces=[]
        spaces = [True if tok.whitespace_ else False for tok in tokens]
        words = [t.text for t in tokens]
        doc = Doc(nlp.vocab, words=words, spaces=spaces)
        
        # print(annotation[0]["result"])
        # Parse the GGP entities
        spans = annotation[0]["result"]
        entities = []
        span_end_to_start = {}
        id_token = {}
        start_span_token = {}
        end_span_token = {}
        # print(spans)
        for span in spans:
            
            if "value" in span.keys():
              # print(span["value"]["start"],span["value"]["end"])
              span_token=nlp(span["value"]["text"])
              len_span_tokens = len(span_token)
              if len_span_tokens > 1 :
                token_start = main_doc_index[span_token[0].text]
                token_end = main_doc_index[span_token[0].text] + len_span_tokens
              else:
                token_start = main_doc_index[span_token[0].text]
                token_end = main_doc_index[span_token[0].text] 

              start_span_token[span["value"]["text"]] = token_start
              # print(token_start, token_end)
              
              entity = doc.char_span(
                    int(span["value"]["start"]), 
                    int(span["value"]["end"]),
                    label = str(span["value"]["labels"][0])
                )


              span_end_to_start[token_start] = token_start
              id_token[span["id"]] = span["value"]["text"]
              # print(span_end_to_start)
              entities.append(entity)
              span_starts.add(token_start)
              

        doc.ents = entities
        # print(doc.ents)
        # print(span_starts)
        # print(id_token)
        # print(start_span_token)

        # Parse the relations
        rels = {}
        for x1 in span_starts:
            for x2 in span_starts:
                rels[(x1, x2)] = {}
        # print(rels)

        #print(len(relations))
        for relation in spans:
          if "from_id" in relation.keys():
            # the 'head' and 'child' annotations refer to the end token in the span
            # but we want the first token
            start =start_span_token[id_token[relation["from_id"]]]
            print(start)
            end= start_span_token[id_token[relation["to_id"]]]
            print(end)
            # start = span_end_to_start[relation["from_id"]]
            # end = span_end_to_start[relation["to_id"]]
            label = relation["labels"][0]
            #print(rels[(start, end)])
            #print(label)
            #label = MAP_LABELS[label]
            # print(start,end)
            if label not in rels[(start, end)]:
                rels[(start, end)][label] = 1.0
                pos += 1
                #print(pos)
                #print(rels[(start, end)])

        # The annotation is complete, so fill in zero's where the data is missing
        for x1 in span_starts:
            for x2 in span_starts:
                for label in MAP_LABELS.values():
                    if label not in rels[(x1, x2)]:
                        neg += 1
                        rels[(x1, x2)][label] = 0.0

                        #print(rels[(x1, x2)])
        doc._.rel = rels
        print(doc._.rel)

        # only keeping documents with at least 1 positive case
        if pos > 0:
                docs["total"].append(doc)
                count_pos["total"] += pos
                count_all["total"] += pos + neg

                
                
print(len(docs["total"]))
print(docs["total"])
docbin = DocBin(docs = docs["total"], store_user_data=True)
docbin.to_disk(train_file)
msg.info(
    f"{len(docs['total'])} training sentences"
)





Paris is in France
3
0
{(0, 0): {'ROLE': 0.0, 'ADDRESS': 0.0}, (0, 3): {'ROLE': 0.0, 'ADDRESS': 0.0}, (3, 0): {'ADDRESS': 1.0, 'ROLE': 0.0}, (3, 3): {'ROLE': 0.0, 'ADDRESS': 0.0}}
Paris is in France
3
0
{(0, 0): {'ROLE': 0.0, 'ADDRESS': 0.0}, (0, 3): {'ROLE': 0.0, 'ADDRESS': 0.0}, (3, 0): {'ADDRESS': 1.0, 'ROLE': 0.0}, (3, 3): {'ROLE': 0.0, 'ADDRESS': 0.0}}
Paris is in France
21
8
35
21
{(8, 8): {'ROLE': 0.0, 'ADDRESS': 0.0}, (8, 35): {'ROLE': 0.0, 'ADDRESS': 0.0}, (8, 21): {'ROLE': 0.0, 'ADDRESS': 0.0}, (35, 8): {'ROLE': 0.0, 'ADDRESS': 0.0}, (35, 35): {'ROLE': 0.0, 'ADDRESS': 0.0}, (35, 21): {'ADDRESS': 1.0, 'ROLE': 0.0}, (21, 8): {'ROLE': 1.0, 'ADDRESS': 0.0}, (21, 35): {'ROLE': 0.0, 'ADDRESS': 0.0}, (21, 21): {'ROLE': 0.0, 'ADDRESS': 0.0}}
Paris is in France
1
11
0
1
{(0, 0): {'ROLE': 0.0, 'ADDRESS': 0.0}, (0, 1): {'ROLE': 1.0, 'ADDRESS': 0.0}, (0, 11): {'ROLE': 0.0, 'ADDRESS': 0.0}, (1, 0): {'ROLE': 0.0, 'ADDRESS': 0.0}, (1, 1): {'ROLE': 0.0, 'ADDRESS': 0.0}, (1, 11): {'ADDRESS': 

In [ ]:
main(ann, train_file, dev_file, test_file)

TypeError: ignored